In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
csv_path = "."
image_path = "./images"
batch_size = 256
epochs = 200
learning_rate = 1e-3
seed = 42

In [3]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
class ImageDataset(Dataset):
    def __init__(self, csv_path, image_path, image_transform=transforms.Compose([transforms.ToTensor()])):
        super(ImageDataset).__init__()
        csv = pd.read_csv(csv_path)
        csv_np = csv.to_numpy()
        
        self.image_transform = image_transform
        self.ids = list(map(str, csv_np[:,0]))
        self.image_path = image_path
        
        self.target = torch.from_numpy(np.array(csv_np[:,[12]], dtype="float")).float()
    
    
    def __getitem__(self,idx):
        image = Image.open(self.image_path+'/'+self.ids[idx]+'.jpg')
        image_tensor = self.image_transform(image)
        image.close()
        return image_tensor, self.target[idx]
    
    
    def __len__(self):
        return len(self.target)

In [5]:
image_transform = transforms.Compose([transforms.ToTensor()
                                     ])

train_dataset = ImageDataset(csv_path+'/train.csv', image_path, image_transform)
test_dataset = ImageDataset(csv_path+'/test.csv', image_path, image_transform)
val_dataset = ImageDataset(csv_path+'/val.csv', image_path, image_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [6]:
class ImageNet(nn.Module):
    def __init__(self):
        super(ImageNet, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=False)
        self.resnet18.fc = nn.Sequential(
        nn.Linear(512, 4096),
        nn.BatchNorm1d(4096),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(4096, 2048),
        nn.BatchNorm1d(2048),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(2048, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(1024, 1)
        )
        self.resnet18.to(device)
    
    def forward(self, x):
        return self.resnet18(x)

In [7]:
model = ImageNet().to(device)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        model.zero_grad()
        prediction = model(X)
        loss = torch.sqrt(criterion(prediction, Y)).to(device)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for X_val, Y_val in val_loader:
            X_val = X_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(X_val)
            val_loss = torch.sqrt(criterion(val_prediction, Y_val)).to(device)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 4672.0181, val_loss: 4667.9331
[Epoch:  2] Average loss: 4594.2466, val_loss: 4500.8057
[Epoch:  3] Average loss: 4313.7705, val_loss: 4658.2295
[Epoch:  4] Average loss: 4401.0811, val_loss: 4434.3462
[Epoch:  5] Average loss: 4852.9316, val_loss: 4656.0889
[Epoch:  6] Average loss: 4582.9897, val_loss: 4380.6406
[Epoch:  7] Average loss: 4678.8623, val_loss: 4659.1743
[Epoch:  8] Average loss: 4507.0454, val_loss: 4354.1768
[Epoch:  9] Average loss: 4662.6431, val_loss: 4677.3232
[Epoch: 10] Average loss: 4395.8877, val_loss: 4854.8730
[Epoch: 11] Average loss: 4425.8467, val_loss: 4390.3994
[Epoch: 12] Average loss: 4279.7222, val_loss: 4214.3857
[Epoch: 13] Average loss: 4285.7705, val_loss: 3965.6694
[Epoch: 14] Average loss: 4289.0293, val_loss: 4168.5547
[Epoch: 15] Average loss: 4323.2402, val_loss: 3423.8088
[Epoch: 16] Average loss: 4301.4980, val_loss: 4816.1924
[Epoch: 17] Average loss: 4532.4521, val_loss: 4541.4810
[Epoch: 18] Average loss: 4463.

[Epoch: 144] Average loss: 1446.8352, val_loss: 3222.3594
[Epoch: 145] Average loss: 1267.6029, val_loss: 3757.5288
[Epoch: 146] Average loss: 1201.0812, val_loss: 3316.7363
[Epoch: 147] Average loss: 1318.7610, val_loss: 3449.1514
[Epoch: 148] Average loss: 1097.8059, val_loss: 3586.0974
[Epoch: 149] Average loss: 1018.3423, val_loss: 3476.0269
[Epoch: 150] Average loss: 1506.7144, val_loss: 3624.1094
[Epoch: 151] Average loss: 1350.4680, val_loss: 3616.3652
[Epoch: 152] Average loss: 1307.3025, val_loss: 3189.4546
[Epoch: 153] Average loss: 1184.3168, val_loss: 3315.2649
[Epoch: 154] Average loss: 1094.5510, val_loss: 3082.5940
[Epoch: 155] Average loss: 2323.4619, val_loss: 3217.8025
[Epoch: 156] Average loss: 1109.8395, val_loss: 3134.5090
[Epoch: 157] Average loss: 1179.8141, val_loss: 3499.4189
[Epoch: 158] Average loss: 1064.5637, val_loss: 3728.2510
[Epoch: 159] Average loss: 1196.9845, val_loss: 3484.8647
[Epoch: 160] Average loss: 1224.0010, val_loss: 3297.6130
[Epoch: 161] A

In [9]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
with torch.no_grad():
    for X_test, Y_test in test_loader:
        X_test = X_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(X_test)
        mean = torch.mean(Y_test)
        ss_tot += torch.sum((Y_test - mean) ** 2)
        ss_res += torch.sum((Y_test - prediction) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 13.80%
